In [36]:
import pandas as pd
from pathlib import Path
import toml
from typing import List
from dataclasses import dataclass, asdict
from IPython.display import Markdown
import codecs

%run _utils.py

def get_jb_files(fdir=Path('.')):
    exts = ['*.ipynb', '*.md']
    fnms = []
    for e in exts: 
        tmp = recursive_glob(rootdir=fdir, pattern=e, recursive=True)
        fnms.extend(tmp)
    fpths = [Path(fnm) for fnm in fnms if ".ipynb_checkpoints" not in fnm]
    names = [p.stem for p in fpths]
    return dict(zip(names,fpths))

def create_img_fpth(tech):
    """creates fpth from name"""
    for t in tech['tech']:
        if t['image'] == '':
            name = t['name']
            t['image'] = f'images/{name}-icon.png'

def create_local_link(tech, paths_map):
    """"""
    for t in tech['tech']:
        if t['local_link'] == '':
            for tag in t['tags']:
                if tag in paths_map.keys():
                    t['local_link'] = paths_map[tag]

@dataclass
class MfTech:
    name: str
    url: str
    description: str
    local_link: str
    image: str

@dataclass
class TechGroup:
    tag: str
    description: str
    local_link: str
    tech: List[MfTech] 


def create_tech_group(df, tag, tag_des) -> TechGroup:

    li_tmp = []
    df_tmp = srch_tgs(df, 'tags', tag, clmn_rtrn=list(df)).reset_index()
    keys = list(MfTech.__dict__['__annotations__'].keys())
    for index, row in df_tmp.iterrows():
        di = { key: dict(row)[key] for key in keys }
        tech = MfTech(**di)
        li_tmp.append(tech)
    if tag in tag_des.keys():
        description = tag_des[tag]
    else:
        description = ""
    local_link = df_tmp.local_link.unique()[0]
    return TechGroup(tag=tag, tech=li_tmp, description=description, local_link=local_link)

def create_tech_groups(df, tags, tag_des):

    li_tech_groups = []
    for tag in tags:
        if type(tag) == list:
            for t in tag:
                li_tech_groups.append(create_tech_group(df, t, tag_des))
        else:
            li_tech_groups.append(create_tech_group(df, tag, tag_des))
    return tuple(li_tech_groups)


def md_img_link(tech: MfTech)-> str:
    return f"[![{tech.name}]({tech.image})]({tech.url} '{tech.description}')"

def md_img_links(grp: TechGroup)-> str:
    return "\n".join([md_img_link(tech) for tech in grp.tech])

def create_panel_obj(grp: TechGroup) -> str:
    imgs = md_img_links(grp)
    return f"""
**[{grp.tag}]({grp.local_link})**
^^^
{grp.description}

{imgs}

---
"""
def create_panel_objs(li_grps: List[TechGroup]) -> str:
    return "\n".join([create_panel_obj(grp) for grp in li_grps])

def create_panel(tu_tech_grps: List[TechGroup]) -> str:
    output = """

:::{panels}
:container: +full-width text-center
:column: col-lg-4 px-2 py-2
:card:

""" + create_panel_objs(tu_tech_grps) + "\n :::"
    
    return output

In [37]:
tags

['object-models',
 'external-experts',
 'deploying',
 'frontend',
 'software-automation',
 'databases',
 ['orm', 'object-models'],
 'packaging']

In [38]:
path_tech = Path('_preferred_technologies.toml')
path_docs = Path('.')

tech = toml.load(path_tech)
create_img_fpth(tech)
paths_map = get_jb_files(path_docs)
create_local_link(tech, paths_map)

df = pd.DataFrame.from_dict(tech['tech'])
tags = list(set(flatten_list(df.tags.tolist())))
tags = [
 'object-models',
 'external-experts',
 #'backend',
 #'developing',
 'deploying',
 'frontend',
 'software-automation',
 'databases',
 ['orm','object-models'],
 'packaging'
]
tag_des = tech['tags']
tu_tech_grps = create_tech_groups(df, tags, tag_des)
md_panels = create_panel(tu_tech_grps)

md_header = """
# mfcode_docs

**Max Fordham Engineering Development Best Practice**

This documentation aims to provide high-level, non-verbose notes outlining Max Fordham's engineering software development infrastructure.
Links to external sources and standards to describe the approach are preferred where possible.
The intended audience are those contributing-to and maintaining internal development tools, or external parties and collaborators interested in our approach.
Code snippets are encouraged.

**We stand on the shoulders of giants**. Our software is built on robust, high-quality opensource packages, and we are proud to share the work that we produce with the opensource community.
When writing code there are always many solutions to a problem;
we standardise the technologies we use for a given task to ensure a consistent approach across projects. See below for key technologies on which we rely.

"""
md = md_header + md_panels 
file = codecs.open("index.md", "w", "utf-8")
file.write(md)
file.close()